<a href="https://colab.research.google.com/github/SimoneFarallo/Thesis/blob/main/Notebook_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Tesi magistrale**

https://it.overleaf.com/project/6502e44ee97a5ee4dea30d17



# Preliminary Operations

In [1]:
!pip install spacy  transformers https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/releases/v0.4.0/en_core_sci_md-0.4.0.tar.gz SentencePiece newspaper stanza

"pip" non � riconosciuto come comando interno o esterno,
 un programma eseguibile o un file batch.


In [2]:
#!pip uninstall protobuf
#!pip install protobuf==3.20.0

In [3]:
!python -m spacy download en_core_web_sm

✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


You should consider upgrading via the 'c:\Users\Simone\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


In [4]:
!pip install stanza

"pip" non � riconosciuto come comando interno o esterno,
 un programma eseguibile o un file batch.


In [1]:
import pandas as pd
import time
import random
import spacy
import torch
from transformers import AutoModel, AutoTokenizer
from transformers import BertTokenizer, BertModel
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer,PorterStemmer
import string
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')
import matplotlib.pyplot as plt
import spacy
import stanza
import os
from stanza.server import CoreNLPClient
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

c:\Users\Simone\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Simone\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Simone\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Simone\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
ls

 Il volume nell'unit� C � Windows
 Numero di serie del volume: BEF8-AC01

 Directory di c:\Users\Simone\Documents\Desktop\Tesi_Magistrale

15/12/2023  19:43    <DIR>          .
15/12/2023  19:43    <DIR>          ..
01/12/2023  11:32                68 .gitattributes
01/12/2023  11:33    <DIR>          Archivio
01/12/2023  11:23    <DIR>          Data
01/12/2023  11:32        24.165.944 Notebook_1.ipynb
15/12/2023  12:15        23.308.521 Notebook_2.ipynb
20/12/2023  12:03        25.569.363 Notebook_3.ipynb
01/12/2023  11:32         4.230.909 output_rebel_triple.txt
18/12/2023  23:32        30.694.482 triple_health.csv
               6 File    107.969.287 byte
               4 Directory   7.724.294.144 byte disponibili


# Dataset Claims
 *  Claims: https://huggingface.co/datasets/health_fact/viewer/default/test

In [16]:
df = pd.read_csv('C:\\Users\\Simone\\Documents\\Desktop\\Tesi_Magistrale\\Data\\train.tsv', sep='\t')
df.head()

,claim_id,claim,date_published,explanation,fact_checkers,main_text,sources,label,subjects
0,15661,"""The money the Clinton Foundation took from fr...","April 26, 2015","""Gingrich said the Clinton Foundation """"took m...",Katie Sanders,"""Hillary Clinton is in the political crosshair...",https://www.wsj.com/articles/clinton-foundatio...,false,"Foreign Policy, PunditFact, Newt Gingrich,"
1,9893,Annual Mammograms May Have More False-Positives,"October 18, 2011",This article reports on the results of a study...,,While the financial costs of screening mammogr...,,mixture,"Screening,WebMD,women's health"
2,11358,SBRT Offers Prostate Cancer Patients High Canc...,"September 28, 2016",This news release describes five-year outcomes...,"Mary Chris Jaklevic,Steven J. Atlas, MD, MPH,K...",The news release quotes lead researcher Robert...,https://www.healthnewsreview.org/wp-content/up...,mixture,"Association/Society news release,Cancer"
3,10166,"Study: Vaccine for Breast, Ovarian Cancer Has ...","November 8, 2011","While the story does many things well, the ove...",,"The story does discuss costs, but the framing ...",http://clinicaltrials.gov/ct2/results?term=can...,true,"Cancer,WebMD,women's health"
4,11276,Some appendicitis cases may not require ’emerg...,"September 20, 2010",We really don’t understand why only a handful ...,,"""Although the story didn’t cite the cost of ap...",,true,


In [17]:
print(len(df))
print(df.columns)

9832
Index(['claim_id', 'claim', 'date_published', 'explanation', 'fact_checkers',
       'main_text', 'sources', 'label', 'subjects'],
      dtype='object')


In [18]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9832 entries, 0 to 9831
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   claim_id        9832 non-null   object
 1   claim           9824 non-null   object
 2   date_published  7895 non-null   object
 3   explanation     9824 non-null   object
 4   fact_checkers   9821 non-null   object
 5   main_text       9806 non-null   object
 6   sources         9804 non-null   object
 7   label           9805 non-null   object
 8   subjects        9804 non-null   object
dtypes: object(9)
memory usage: 691.4+ KB


## Dimensionality Reduction GPT - 3.5

The prompt was run directly from the interface via prompt engeeniring

In [19]:
df[['claim','subjects','label']]

,claim,subjects,label
0,"""The money the Clinton Foundation took from fr...","Foreign Policy, PunditFact, Newt Gingrich,",false
1,Annual Mammograms May Have More False-Positives,"Screening,WebMD,women's health",mixture
2,SBRT Offers Prostate Cancer Patients High Canc...,"Association/Society news release,Cancer",mixture
3,"Study: Vaccine for Breast, Ovarian Cancer Has ...","Cancer,WebMD,women's health",true
4,Some appendicitis cases may not require ’emerg...,,true
...,...,...,...
9827,The Sturgis motorcycle rally in 2020 resulted ...,"Politics Medical, COVID-19",unproven
9828,AstraZeneca's infant respiratory drug prioriti...,Health News,true
9829,Testicular cancer deaths double with after 40 ...,Health News,true
9830,The FDA published “conclusive proof” that the...,Medical,false


In [20]:
# Converti tutti i valori nella colonna 'claim' in stringhe
df['claim'] = df['claim'].astype(str)
df['subjects'] = df['subjects'].astype(str)

In [21]:
kw_uniche = df["subjects"].unique()
print(kw_uniche)
print(len(kw_uniche))

['Foreign Policy, PunditFact, Newt Gingrich, '
 "Screening,WebMD,women's health"
 'Association/Society news release,Cancer' ...
 'Public Health, Florida, Carlos Gimenez, ' 'breast cancer,red wine'
 'National, Economy, Chris Dodd, ']
5812


In [22]:
# Definire le parole chiave per l'ambito medico
parole_chiave_mediche = keywords = ['medical', 'biomedical', 'drug', 'health', 'disease', 'coronavirus', 'cancer', 'research', 'hospital', 'virus', 'pharmaceutical', 'clinical', 'therapy', 'treatment','Screening', "women's health", 'Health News', 'platelet-rich plasma', 'Virus Outbreak', 'Public health', 'Addiction treatment', 'Medicaid', 'stem cells', 'Coronavirus', 'flu', 'Health care', 'immunotherapy', "Alzheimer's disease", 'Imaging', 'Muscular dystrophy', 'Marijuana', 'Mental health', 'Heart disease', 'Diabetes', 'acupuncture', 'HIV', 'Abortion', 'National Institutes of Health (NIH)', 'Medical marijuana', 'Civil Rights', 'Gays and Lesbians', 'Sexuality', 'Disabilities', 'Disability', 'Tuberculosis', 'Cardiovascular disease']

# Filtrare le righe che contengono le parole chiave
filtro = df["subjects"].str.contains('|'.join(parole_chiave_mediche), case=False) | df["claim"].str.contains('|'.join(parole_chiave_mediche), case=False)
df = df[filtro]

# Stampare i risultati
df

C:\Users\Simone\AppData\Local\Temp\ipykernel_19012\923370402.py:5: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  filtro = df["subjects"].str.contains('|'.join(parole_chiave_mediche), case=False) | df["claim"].str.contains('|'.join(parole_chiave_mediche), case=False)


,claim_id,claim,date_published,explanation,fact_checkers,main_text,sources,label,subjects
1,9893,Annual Mammograms May Have More False-Positives,"October 18, 2011",This article reports on the results of a study...,,While the financial costs of screening mammogr...,,mixture,"Screening,WebMD,women's health"
2,11358,SBRT Offers Prostate Cancer Patients High Canc...,"September 28, 2016",This news release describes five-year outcomes...,"Mary Chris Jaklevic,Steven J. Atlas, MD, MPH,K...",The news release quotes lead researcher Robert...,https://www.healthnewsreview.org/wp-content/up...,mixture,"Association/Society news release,Cancer"
3,10166,"Study: Vaccine for Breast, Ovarian Cancer Has ...","November 8, 2011","While the story does many things well, the ove...",,"The story does discuss costs, but the framing ...",http://clinicaltrials.gov/ct2/results?term=can...,true,"Cancer,WebMD,women's health"
5,8713,Britain to reveal trial criteria for coronavir...,"April 7, 2020",British regulators will this week reveal appro...,Alistair Smout,Antibody tests show whether whether people hav...,,true,Health News
7,2768,U.S. says results encouraging for healthcare d...,"January 30, 2014",The Obama administration on Thursday reported ...,David Morgan,As part of President Barack Obama’s healthcare...,,true,Health News
...,...,...,...,...,...,...,...,...,...
9826,8197,Venezuela expands quarantine as number of coro...,"March 16, 2020",Venezuela will implement a nationwide quaranti...,"Angus Berwick, Luc Cohen, Mariela Nava",The country began a quarantine on Monday in a ...,,true,Health News
9827,35948,The Sturgis motorcycle rally in 2020 resulted ...,"September 10, 2020","They want to know if mass-events (protests, co...",Dan Evon,"In September 2020, social media was abuzz over...",,unproven,"Politics Medical, COVID-19"
9828,401,AstraZeneca's infant respiratory drug prioriti...,"September 25, 1995",Britain’s AstraZeneca said a potential medicin...,,The “Breakthrough Therapy” and “Prime” designa...,,true,Health News
9829,2023,Testicular cancer deaths double with after 40 ...,"February 10, 2011",Men diagnosed with testicular cancer at 40 yea...,,This was true even when initial treatment and ...,http://bit.ly/fGNEw9,true,Health News


# Preprocessing text

In [23]:
# Converti tutti i valori nella colonna 'claim' in stringhe
df['claim'] = df['claim'].astype(str)
df['subjects'] = df['subjects'].astype(str)

# Estrazione triple REBEL

In [25]:
# Load model and tokenizer
tokenizer = AutoTokenizer.from_pretrained("Babelscape/rebel-large")
model = AutoModelForSeq2SeqLM.from_pretrained("Babelscape/rebel-large")
gen_kwargs = {
    "max_length": 256,
    "length_penalty": 0,
    "num_beams": 3,
    "num_return_sequences": 3,
}

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [26]:
def extract_triplets(text):
    triplets = []
    relation, subject, relation, object_ = '', '', '', ''
    text = text.strip()
    current = 'x'
    for token in text.replace("<s>", "").replace("<pad>", "").replace("</s>", "").split():
        if token == "<triplet>":
            current = 't'
            if relation != '':
                triplets.append({'head': subject.strip(), 'type': relation.strip(),'tail': object_.strip()})
                relation = ''
            subject = ''
        elif token == "<subj>":
            current = 's'
            if relation != '':
                triplets.append({'head': subject.strip(), 'type': relation.strip(),'tail': object_.strip()})
            object_ = ''
        elif token == "<obj>":
            current = 'o'
            relation = ''
        else:
            if current == 't':
                subject += ' ' + token
            elif current == 's':
                object_ += ' ' + token
            elif current == 'o':
                relation += ' ' + token
    if subject != '' and relation != '' and object_ != '':
        triplets.append({'head': subject.strip(), 'type': relation.strip(),'tail': object_.strip()})
    return triplets


In [27]:
# Function to apply
def apply_extraction(row):
    # Tokenize the text
    model_inputs = tokenizer(row['claim'], max_length=256, padding=True, truncation=True, return_tensors='pt')



    # Generate
    generated_tokens = model.generate(
        model_inputs["input_ids"].to(model.device),
        attention_mask=model_inputs["attention_mask"].to(model.device),
        **gen_kwargs,
    )

    # Decode
    decoded_preds = tokenizer.batch_decode(generated_tokens, skip_special_tokens=False)

    # Extract triplets
    triplets = []
    for idx, sentence in enumerate(decoded_preds):
        triplets.extend(extract_triplets(sentence))

    return triplets

# Apply the function to each row in the DataFrame
df['triplets'] = df.apply(apply_extraction, axis=1)

# Display the results
for index, row in df.iterrows():
    print(f"Claim : {row['claim']}")
    print(f"Triplets: {row['triplets']}")
    print("-" * 50)

Claim : Annual Mammograms May Have More False-Positives
Triplets: [{'head': 'False-Positives', 'type': 'facet of', 'tail': 'Annual Mammograms'}, {'head': 'False-Positives', 'type': 'part of', 'tail': 'Annual Mammograms'}, {'head': 'False-Positives', 'type': 'subclass of', 'tail': 'False-Positives'}]
--------------------------------------------------
Claim : SBRT Offers Prostate Cancer Patients High Cancer Control and Low Toxicity in Fewer Treatments
Triplets: [{'head': 'SBRT', 'type': 'medical condition treated', 'tail': 'Prostate Cancer'}, {'head': 'SBRT', 'type': 'medical condition treated', 'tail': 'Prostate Cancer'}, {'head': 'Prostate Cancer', 'type': 'drug used for treatment', 'tail': 'SBRT'}, {'head': 'SBRT', 'type': 'facet of', 'tail': 'Prostate Cancer'}]
--------------------------------------------------
Claim : Study: Vaccine for Breast, Ovarian Cancer Has Potential
Triplets: [{'head': 'Ovarian Cancer', 'type': 'has part', 'tail': 'Breast'}, {'head': 'Breast', 'type': 'has ef

Tempo di esecuzione: 21683.98

In [28]:
df.head()

,claim_id,claim,date_published,explanation,fact_checkers,main_text,sources,label,subjects,triplets
1,9893,Annual Mammograms May Have More False-Positives,"October 18, 2011",This article reports on the results of a study...,,While the financial costs of screening mammogr...,,mixture,"Screening,WebMD,women's health","[{'head': 'False-Positives', 'type': 'facet of..."
2,11358,SBRT Offers Prostate Cancer Patients High Canc...,"September 28, 2016",This news release describes five-year outcomes...,"Mary Chris Jaklevic,Steven J. Atlas, MD, MPH,K...",The news release quotes lead researcher Robert...,https://www.healthnewsreview.org/wp-content/up...,mixture,"Association/Society news release,Cancer","[{'head': 'SBRT', 'type': 'medical condition t..."
3,10166,"Study: Vaccine for Breast, Ovarian Cancer Has ...","November 8, 2011","While the story does many things well, the ove...",,"The story does discuss costs, but the framing ...",http://clinicaltrials.gov/ct2/results?term=can...,true,"Cancer,WebMD,women's health","[{'head': 'Ovarian Cancer', 'type': 'has part'..."
5,8713,Britain to reveal trial criteria for coronavir...,"April 7, 2020",British regulators will this week reveal appro...,Alistair Smout,Antibody tests show whether whether people hav...,,true,Health News,"[{'head': 'coronavirus', 'type': 'has effect',..."
7,2768,U.S. says results encouraging for healthcare d...,"January 30, 2014",The Obama administration on Thursday reported ...,David Morgan,As part of President Barack Obama’s healthcare...,,true,Health News,"[{'head': 'healthcare delivery reforms', 'type..."


In [29]:
#df.to_csv('triple_health.csv')